In [0]:
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.callbacks import TensorBoard
import pandas as pd
import numpy as np
from datetime import datetime
import tensorflow as tf

In [0]:
# model
class MLP(tf.keras.Model):
    def __init__(self):
        super(MLP, self).__init__()
        self.dense1 = tf.keras.layers.Dense(512, activation="relu")
        self.dense2 = tf.keras.layers.Dense(512, activation="relu")
        self.dropout1 = tf.keras.layers.Dropout(0.1)
        self.dense3 = tf.keras.layers.Dense(1028, activation="relu")
        self.dense4 = tf.keras.layers.Dense(1, activation="relu")

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dropout1(x)
        x = self.dense3(x)
        return self.dense4(x)

In [0]:
# 0 ~ 1 の範囲に正則化
def min_max(x):
    min_value = x.min()
    max_value = x.max()
    result = (x - min_value)/(max_value - min_value)
    return result

def create_feature(data):
    feature = {}
    cat_cols = ['駅名', '建物構造', '間取り']
    for cat_col in cat_cols:
        cat_data = data[cat_col].values
        cat_categories = data[cat_col].unique().tolist()
        cat_indices = []
        for cat_datum in cat_data:
            cat_indices.append(cat_categories.index(cat_datum))
        feature[cat_col] = np.eye(len(cat_categories))[cat_indices]

    feature["所在階"] = min_max(data["所在階"].str.replace('B', '-').astype(int).values) #地下表記を数字に変換
    feature["徒歩分"] = min_max(data["徒歩分"].values)
    feature["専有面積"] = min_max(data["専有面積"].values)
    feature["部屋数"] = min_max(data["部屋数"].values)
    feature["築年"] = data["築年"].values
    feature["築年"] = pd.to_datetime(data["築年"].astype(str), format="%Y%M", errors="coerce")
    feature["築年"] = min_max((pd.to_datetime("2019-10-01") - feature["築年"]).dt.days.values.astype(int))

    feature_vec = np.hstack((
        feature["駅名"],
        feature["建物構造"],
        feature["間取り"],
        feature["徒歩分"][:, np.newaxis],
        feature["専有面積"][:, np.newaxis],
        feature["部屋数"][:, np.newaxis],
        feature["築年"][:, np.newaxis],
        feature["所在階"][:, np.newaxis],
    ))

    return feature_vec

In [0]:
data = pd.read_csv("dummy_data.csv")

In [0]:
#train test split
X = create_feature(data)
Y = data["成約価格"].values
x_train, x_test, y_train, y_test= train_test_split(X, Y, test_size=5000, random_state=0)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=5000, random_state=0)

In [0]:
# train
model = MLP()
tensorboard = TensorBoard(log_dir="logs")
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "./checkpoint/MLP-{epoch:04d}.ckpt",
    verbose=1,
    save_weights_only=True,
    period=500
)
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-1),
    loss='mean_absolute_percentage_error',
    metrics=[
        "mean_absolute_percentage_error", 
 #       "mean_absolute_error",
 #       "mean_squared_error"
    ]
)
model.fit(
    x_train, 
    y_train, 
    epochs=100,
    batch_size=512,
    validation_data=(x_valid, y_valid),
    callbacks=[tensorboard, checkpoint]
)

Train on 30000 samples, validate on 5000 samples
Epoch 1/100
30000/30000 [==============================] - 3s 94us/sample - loss: 63.4185 - mean_absolute_percentage_error: 63.4185 - val_loss: 29.3284 - val_mean_absolute_percentage_error: 29.3284
Epoch 2/100
30000/30000 [==============================] - 3s 89us/sample - loss: 13.7878 - mean_absolute_percentage_error: 13.7878 - val_loss: 9.4491 - val_mean_absolute_percentage_error: 9.4491
Epoch 3/100
30000/30000 [==============================] - 3s 89us/sample - loss: 9.5135 - mean_absolute_percentage_error: 9.5135 - val_loss: 9.2566 - val_mean_absolute_percentage_error: 9.2566
Epoch 4/100
30000/30000 [==============================] - 3s 89us/sample - loss: 9.2067 - mean_absolute_percentage_error: 9.2067 - val_loss: 8.8954 - val_mean_absolute_percentage_error: 8.8954
Epoch 5/100
30000/30000 [==============================] - 3s 88us/sample - loss: 8.9930 - mean_absolute_percentage_error: 8.9930 - val_loss: 9.5917 - val_mean_absolute_

In [0]:
train_mape, train_acc = model.evaluate(x_train,  y_train)
val_mape, val_acc = model.evaluate(x_valid,  y_valid)
print(f'train mape : {train_mape:.3f}%' )

print(f'valid mape : {val_mape:.3f}%' )

5000/5000 [==============================] - 0s 77us/sample - loss: 7.6253 - mean_absolute_percentage_error: 7.6253
train mape : 6.573%
valid mape : 7.625%


In [0]:
test_mape, test_acc = model.evaluate(x_test,  y_test)
print(f'test mape : {test_mape:.3f}%' )

5000/5000 [==============================] - 0s 78us/sample - loss: 7.3816 - mean_absolute_percentage_error: 7.3816
test mape : 7.382%


In [0]:
model.dropout1 = tf.keras.layers.Dropout(0.0)
data = data.append(pd.DataFrame.from_dict({
    "駅名": ["新橋"],
    "建物構造": ["SRC"],
    "徒歩分": [10],
    "専有面積": [30],
    "間取り": ["R"],
    "部屋数": [1],
    "築年": ["198001"],
    "所在階": ["5"],
    "成約価格": [-1]
})).reset_index(drop=True)
X = create_feature(data)
model.predict_on_batch(X[-1:])

array([[19491340.]], dtype=float32)

In [0]:
model.dropout1 = tf.keras.layers.Dropout(0.0)
data = data.append(pd.DataFrame.from_dict({
    "駅名": ["新橋"],
    "建物構造": ["SRC"],
    "徒歩分": [10],
    "専有面積": [30],
    "間取り": ["R"],
    "部屋数": [1],
    "築年": ["198001"],
    "所在階": ["5"],
    "成約価格": [-1]
})).reset_index(drop=True)
X = create_feature(data)
model.predict_on_batch(X[-1:])

array([[19491340.]], dtype=float32)